This notebook is part the of Dr. Christoforos Christoforou's course materials. You may not, nor may you knowingly allow others to reproduce or distribute lecture notes, course materials or any of their derivatives without the instructor's express written consent.

## Problem Set 09: Practical Consideration in CNN

For this problem set, you are expected to load an external dataset, and build a convolutional neural network that classifies images of Cat and Dogs. You are expected to implement the network based on the practical considerations outlined in the corresponding lecture. REview the video lecture of this course to complete the assignment. You wull need the following libraries to complete this probelem set.

In [166]:
# Load general numpy and pandas 
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd

# Load tensorflow libraries 
import tensorflow 
from tensorflow.keras import backend


# Dataset Libraries 
from tensorflow.keras.datasets import fashion_mnist 

In [167]:
import os
import tensorflow.keras 
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Conv2D, MaxPool2D, BatchNormalization, Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input

## Add any missing libraries 

## Exercise

Create a convolutional neural network to classify images in the above dataset to either Cat or Dogs. Your solution should comply with the following requirements:

* Build computational blocks that include a sqeunce of `Covolution Layer`-> `BatchNormailization` -> `Activation` -> `Max Pooling` -> `Dropout Layer`.
* Build computational blocks that include a sequence of `Dense Layer`-> `BatchNormailization Layer` -> `Activation function` -> `Max Pooling`.
* Build a model that utilizes the above computational blocks. You can use additional layers if you like.
* Create a manual split of the dataset set into a `Training Set` and `Validation Set`. You are expected to store the training and validation set into separate filders so they can be loaded using the `ImageDataGenerator` class.
* Prepare `ImageDataGenerator` objects to load batches of training and validation images from the corresponding folders. Add an image augmentation layer and make sure you use the `flow_from_directory` method.
* Train the model using `EarlyStopping` and `ModelCheckpoint` callback functions.
* Report validation performance on each epoch.
* Fine-tune your model to achieve classification performance above 80%.
* Save the best model.
* Load the best model and display the model structure (i.e. using the `model.summary()` method. 


Use as many cells as you need to complete the problem set.


Copyright Statement: Copyright © 2020 Christoforou. The materials provided by the instructor of this course, including this notebook, are for the use of the students enrolled in the course. Materials are presented in an educational context for personal use and study and should not be shared, distributed, disseminated or sold in print — or digitally — outside the course without permission. You may not, nor may you knowingly allow others to reproduce or distribute lecture notes, course materials as well as any of their derivatives without the instructor's express written consent.

In [168]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
zip_dir = tf.keras.utils.get_file('cats_and_dogs_filterted.zip', origin=_URL, extract=True)

In [169]:
base_dir = os.path.join(os.path.dirname(zip_dir), 'cats_and_dogs_filtered')
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
train_cats_dir = os.path.join(train_dir, 'cats')  
train_dogs_dir = os.path.join(train_dir, 'dogs')  
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')  

In [ ]:
zip_dir_base = os.path.dirname(zip_dir)
!find $zip_dir_base -type d

In [193]:
BATCH_SIZE = 30
img_shape  = 150  

In [194]:
train_image_generator      = ImageDataGenerator(rescale=1./255) 
validation_image_generator = ImageDataGenerator(rescale=1./255) 

In [195]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=BATCH_SIZE,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(img_shape,img_shape), 
                                                           class_mode='binary')

Found 2000 images belonging to 2 classes.


In [196]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=BATCH_SIZE,
                                                              directory=validation_dir,
                                                              shuffle=False,
                                                              target_size=(img_shape,img_shape), 
                                                              class_mode='binary')

Found 1000 images belonging to 2 classes.


In [197]:
sample_training_images, _ = next(train_data_gen) 

In [198]:
model = Sequential()

model.add(Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(MaxPool2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dense(2))


In [205]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_28 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_52 (MaxPooling (None, 74, 74, 32)        0         
_________________________________________________________________
activation_43 (Activation)   (None, 74, 74, 32)        0         
_________________________________________________________________
max_pooling2d_53 (MaxPooling (None, 37, 37, 32)        0         
_________________________________________________________________
dropout_19 (Dropout)         (None, 37, 37, 32)        0         
_________________________________________________________________
max_pooling2d_54 (MaxPooling (None, 18, 18, 32)        0         
_________________________________________________________________
flatten_22 (Flatten)         (None, 10368)           

In [208]:
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=int(np.ceil(total_train / float(BATCH_SIZE))),
    epochs=15,
    validation_data=val_data_gen,
    validation_steps=int(np.ceil(total_val / float(BATCH_SIZE))),
    callbacks=[callback]
)


Epoch 1/15
67/67 [==============================] - 38s 574ms/step - loss: 0.4561 - accuracy: 0.7980 - val_loss: 0.6147 - val_accuracy: 0.6610
Epoch 2/15
67/67 [==============================] - 38s 572ms/step - loss: 0.3980 - accuracy: 0.8200 - val_loss: 0.5898 - val_accuracy: 0.6870
Epoch 3/15
67/67 [==============================] - 38s 569ms/step - loss: 0.3644 - accuracy: 0.8435 - val_loss: 0.5540 - val_accuracy: 0.7190
Epoch 4/15
67/67 [==============================] - 40s 591ms/step - loss: 0.2940 - accuracy: 0.8795 - val_loss: 0.5671 - val_accuracy: 0.7080
Epoch 5/15
67/67 [==============================] - 39s 575ms/step - loss: 0.2635 - accuracy: 0.8945 - val_loss: 0.6050 - val_accuracy: 0.6850
Epoch 6/15
67/67 [==============================] - 39s 576ms/step - loss: 0.2373 - accuracy: 0.9090 - val_loss: 0.6272 - val_accuracy: 0.6990
Epoch 7/15
67/67 [==============================] - 38s 574ms/step - loss: 0.2132 - accuracy: 0.9190 - val_loss: 0.6050 - val_accuracy: 0.7110

In [209]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

print("Accuracy" , acc, '\n')
print("Value Accuracy", val_acc, '\n')
print("Loss", loss, '\n')
print("Value Loss", val_loss, '\n')
print("Epoch Range", epochs_range, 'n')

Accuracy [0.7979999780654907, 0.8199999928474426, 0.843500018119812, 0.8794999718666077, 0.8945000171661377, 0.9089999794960022, 0.9190000295639038, 0.934499979019165, 0.9474999904632568, 0.9589999914169312, 0.9465000033378601, 0.9639999866485596, 0.9760000109672546, 0.9695000052452087, 0.9775000214576721] 

Value Accuracy [0.6610000133514404, 0.6869999766349792, 0.718999981880188, 0.7080000042915344, 0.6850000023841858, 0.6990000009536743, 0.7110000252723694, 0.718999981880188, 0.7120000123977661, 0.7099999785423279, 0.6919999718666077, 0.6990000009536743, 0.6980000138282776, 0.703000009059906, 0.6990000009536743] 

Loss [0.45606499910354614, 0.39799603819847107, 0.3643592298030853, 0.2939690053462982, 0.26345202326774597, 0.23725271224975586, 0.21324415504932404, 0.17471668124198914, 0.1356980949640274, 0.11967170983552933, 0.13928097486495972, 0.10056712478399277, 0.08100325614213943, 0.07885374873876572, 0.06874213367700577] 

Value Loss [0.6146973371505737, 0.5897642970085144, 0.5